# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

In [ ]:
# set environment in colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"
!pip install pyspark


     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 198 kB 34.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=e6bf0d848edc26fb4792f8095139700cfece40fe0d90834787adecb1959cb734
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Colab").config("spark.ui.port", "4050").getOrCreate()

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

housing = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("data/housing.csv")
housing.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

housing.count()

20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Print the number of records with population more than 10000.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

housing.filter("population > 10000").show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -121.92|   37.53|               7.0|    28258.0|        3864.0|   12203.0|    3701.0|       8.4045|          451100.0|      <1H OCEAN|
|  -117.78|   34.03|               8.0|    32054.0|        5290.0|   15507.0|    5050.0|       6.0191|          253900.0|      <1H OCEAN|
|  -117.87|   34.04|               7.0|    27700.0|        4179.0|   15037.0|    4072.0|       6.6288|          339700.0|      <1H OCEAN|
|  -117.88|   33.96|              16.0|    19059.0|        3079.0|   10988.0|    3061.0|       5.5469|          265200.0|      <1H OCEAN|
|  -118.78|   34.16|              

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

housing.describe(["housing_median_age"]).show()
housing.describe(["total_rooms"]).show()
housing.describe(["median_house_value"]).show()
housing.describe(["population"]).show()

+-------+------------------+
|summary|housing_median_age|
+-------+------------------+
|  count|             20640|
|   mean|28.639486434108527|
| stddev| 12.58555761211163|
|    min|               1.0|
|    max|              52.0|
+-------+------------------+

+-------+------------------+
|summary|       total_rooms|
+-------+------------------+
|  count|             20640|
|   mean|2635.7630813953488|
| stddev|2181.6152515827944|
|    min|               2.0|
|    max|           39320.0|
+-------+------------------+

+-------+------------------+
|summary|median_house_value|
+-------+------------------+
|  count|             20640|
|   mean|206855.81690891474|
| stddev|115395.61587441359|
|    min|           14999.0|
|    max|          500001.0|
+-------+------------------+

+-------+------------------+
|summary|        population|
+-------+------------------+
|  count|             20640|
|   mean|1425.4767441860465|
| stddev|  1132.46212176534|
|    min|               3.0|
|    max|  

Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.sql.functions import max, min, avg

housing.select(max("housing_median_age"), min("total_rooms"), avg("median_house_value")).show()

+-----------------------+----------------+-----------------------+
|max(housing_median_age)|min(total_rooms)|avg(median_house_value)|
+-----------------------+----------------+-----------------------+
|                   52.0|             2.0|     206855.81690891474|
+-----------------------+----------------+-----------------------+



## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity").count().sort("count", ascending = False).show()

+---------------+-----+
|ocean_proximity|count|
+---------------+-----+
|      <1H OCEAN| 9136|
|         INLAND| 6551|
|     NEAR OCEAN| 2658|
|       NEAR BAY| 2290|
|         ISLAND|    5|
+---------------+-----+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

housing.groupBy("ocean_proximity").agg(avg("median_house_value").alias("avg_value")).show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, avg(median_house_value) as avg_value FROM df GROUP by ocean_proximity").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.feature import VectorAssembler

va = VectorAssembler().setInputCols(["housing_median_age", "total_rooms", "median_house_value", "population"])\
                      .setOutputCol("Correlation")

housingAttrs = va.transform(housing)

housingAttrs.select("Correlation").show(5, truncate = False)

+-----------------------------+
|Correlation                  |
+-----------------------------+
|[41.0,880.0,452600.0,322.0]  |
|[21.0,7099.0,358500.0,2401.0]|
|[52.0,1467.0,352100.0,496.0] |
|[52.0,1274.0,341300.0,558.0] |
|[52.0,1627.0,342200.0,565.0] |
+-----------------------------+
only showing top 5 rows



In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.linalg import Matrix
from pyspark.ml.stat import Correlation
from pyspark.sql import Row

row = Correlation.corr(housingAttrs, "Correlation").head()

print("The standard correlation coefficient:\n" + str(row[0]))

The standard correlation coefficient:
DenseMatrix([[ 1.        , -0.3612622 ,  0.10562341, -0.29624424],
             [-0.3612622 ,  1.        ,  0.13415311,  0.85712597],
             [ 0.10562341,  0.13415311,  1.        , -0.02464968],
             [-0.29624424,  0.85712597, -0.02464968,  1.        ]])


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = total_bedrooms / total_rooms
population_per_household = population / households
```

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

housingCol1 = housing.withColumn("rooms_per_household", housing.total_rooms / housing.households)
housingCol2 = housingCol1.withColumn("bedrooms_per_room", housing.total_bedrooms / housing.total_rooms)
housingExtra = housingCol2.withColumn("population_per_household", housing.population / housing.households)

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

renamedHousing = housingExtra.withColumnRenamed("median_house_value","label")
renamedHousing.show(1)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
only showing top 1 row



Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [ ]:
# label columns
colLabel = "label"

# categorical columns
colCat = "ocean_proximity"

# numerical columns
colNum = renamedHousing.schema.names
colNum.remove(colLabel)
colNum.remove(colCat)
print(colNum)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'rooms_per_household', 'bedrooms_per_room', 'population_per_household']


## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
from pyspark.sql.functions import isnan, when, count, col

renamedHousing.select([count(when(col(c).isNull(), c)).alias(c) for c in renamedHousing.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-----+---------------+-------------------+-----------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|label|ocean_proximity|rooms_per_household|bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-----+---------------+-------------------+-----------------+------------------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|    0|              0|                  0|              207|                       0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+-----+---------------+-------------------+-----------------+------------------------+



As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.feature import Imputer

imputer = Imputer().setStrategy("median").setInputCols(["total_bedrooms", "bedrooms_per_room"])\
                                         .setOutputCols(["total_bedrooms", "bedrooms_per_room"])

imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)

imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows



### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.feature import VectorAssembler, StandardScaler

va = VectorAssembler().setInputCols(colNum).setOutputCol("vector_of_features")

featuredHousing = va.transform(imputedHousing)

scaler = StandardScaler().setInputCol("vector_of_features").setOutputCol("scaled_features")
scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|  vector_of_features|     scaled_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-61.007269596069...|
|  -122.22|   37.86|              21.0|     7099.0|        1

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
from pyspark.sql.functions import countDistinct

renamedHousing.select("ocean_proximity").distinct().count()

5

### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer().setInputCol("ocean_proximity").setOutputCol("indexed_ocean_proximity")
idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|indexed_ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                    3.0|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.2

Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [ ]:
indexer.fit(renamedHousing).labels

['<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'NEAR BAY', 'ISLAND']

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder().setInputCol("indexed_ocean_proximity").setOutputCol("encoded_ocean_proximity")
ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|indexed_ocean_proximity|encoded_ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+-----------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|                    3.0|          (4,[3],[1.0])|
|  -122.22|   37.86|              21

---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml import Pipeline

numPipeline = Pipeline().setStages([imputer, va, scaler])
catPipeline = Pipeline().setStages([indexer, encoder])
pipeline = Pipeline().setStages([numPipeline, catPipeline])
newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-----------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|  vector_of_features|     scaled_features|indexed_ocean_proximity|encoded_ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+-----------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

va2 = VectorAssembler().setInputCols(["scaled_features", "encoded_ocean_proximity"]).setOutputCol("features")
dataset = va2.transform(newHousing).select("features", "label")

dataset.show(5)

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-61.007269596069...|452600.0|
|[-61.002278409814...|358500.0|
|[-61.012260782324...|352100.0|
|[-61.017251968579...|341300.0|
|[-61.017251968579...|342200.0|
+--------------------+--------+
only showing top 5 rows



---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

trainSet, testSet = dataset.randomSplit([0.8, 0.2])

## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.regression import LinearRegression

# train the model
lr = LinearRegression().setFeaturesCol("features").setLabelCol("label")
lrModel = lr.fit(trainSet)
trainingSummary = lrModel.summary

print("Coefficients: " + str(lrModel.coefficients) + ", Intercept: " + str(lrModel.intercept))
print("RMSE: " + str(trainingSummary.rootMeanSquaredError))

Coefficients: [-57147.737985118554,-57935.70791035172,13455.483250805146,6750.686124093028,3874.5347931340953,-48022.481438327224,42155.64259729657,77640.31302678415,5806.120200490973,15768.43875178104,3449.9584552181536,-140749.52706638593,-175089.93329012993,-136995.1252249268,-144198.43778972188], Intercept: -2353889.082927693
RMSE: 68282.61822335226


Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.evaluation import RegressionEvaluator

# make predictions on the test data
predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error.
evaluator = RegressionEvaluator().setPredictionCol("prediction").setLabelCol("label").setMetricName("rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = " + str(rmse))

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|201051.75513811782| 94600.0|[-62.065401082150...|
|103525.83735515922| 85800.0|[-62.040445150874...|
|114319.81307244767| 73200.0|[-62.005506847089...|
| 160615.2624858003| 70200.0|[-61.980550915813...|
| 224187.4537583082|116100.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 66593.54990756433


## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

dt = DecisionTreeRegressor().setFeaturesCol("features").setLabelCol("label")

# train the model
dtModel = dt.fit(trainSet)

# make predictions on the test data
predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error
evaluator = RegressionEvaluator().setPredictionCol("prediction").setLabelCol("label").setMetricName("rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = " + str(rmse))

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|176350.12317961166| 94600.0|[-62.065401082150...|
|145757.10947503202| 85800.0|[-62.040445150874...|
|145757.10947503202| 73200.0|[-62.005506847089...|
|145757.10947503202| 70200.0|[-61.980550915813...|
|244505.91454730416|116100.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 66944.91782491452


## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

rf = RandomForestRegressor().setFeaturesCol("features").setLabelCol("label")

# train the model
rfModel = rf.fit(trainSet)

# make predictions on the test data
predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error
evaluator = RegressionEvaluator().setPredictionCol("prediction").setLabelCol("label").setMetricName("rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = " + str(rmse))

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|195947.84431306663| 94600.0|[-62.065401082150...|
|183631.78259895332| 85800.0|[-62.040445150874...|
|198440.02599346772| 73200.0|[-62.005506847089...|
|173560.95241840873| 70200.0|[-61.980550915813...|
|242832.88030477715|116100.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 64200.613375867004


## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

gb = GBTRegressor().setFeaturesCol("features").setLabelCol("label")

# train the model
gbModel = gb.fit(trainSet)

# make predictions on the test data
predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error
evaluator = RegressionEvaluator().setPredictionCol("prediction").setLabelCol("label").setMetricName("rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = " + str(rmse))

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 152435.2306188349| 94600.0|[-62.065401082150...|
| 84370.03048787032| 85800.0|[-62.040445150874...|
|102754.79100384345| 73200.0|[-62.005506847089...|
| 81681.91756661709| 70200.0|[-61.980550915813...|
|160190.61269088596|116100.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 55034.80320792159


---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator

paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [1, 5, 10]).addGrid(rf.maxDepth, [5, 10, 15]).build()

evaluator = RegressionEvaluator().setLabelCol("label").setPredictionCol("prediction").setMetricName("rmse")
cv = CrossValidator().setEstimator(rf).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(3)
cvModel = cv.fit(trainSet)

predictions = cvModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = " + str(rmse))

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|184486.36363636362| 94600.0|[-62.065401082150...|
| 79155.64194373402| 85800.0|[-62.040445150874...|
|123204.34782608696| 73200.0|[-62.005506847089...|
| 78914.44444444445| 70200.0|[-61.980550915813...|
|          198530.0|116100.0|[-61.975559729558...|
+------------------+--------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 51357.24050647177


---
# 7. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

## 7.1. Load the data

In [ ]:
ccdefault = spark.read.format("csv").option("inferSchema", "true").option("header", "true").\
            load("data/ccdefault.csv")
ccdefault.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|     3261|       0|    1000|    1000|    1000|       0|    2000|    

## 7.2. Carry out some exploratory analyses

Print the schema of the dataset

In [ ]:
ccdefault.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- DEFAULT: integer (nullable = tru

Print the number of records in the dataset.

In [ ]:
ccdefault.count()

30000

Print a summary of the table statistics for the attributes `LIMIT_BAL` and `AGE`.

In [ ]:
ccdefault.describe(["LIMIT_BAL"]).show()
ccdefault.describe(["AGE"]).show()

+-------+------------------+
|summary|         LIMIT_BAL|
+-------+------------------+
|  count|             30000|
|   mean|167484.32266666667|
| stddev|129747.66156720246|
|    min|             10000|
|    max|           1000000|
+-------+------------------+

+-------+-----------------+
|summary|              AGE|
+-------+-----------------+
|  count|            30000|
|   mean|          35.4855|
| stddev|9.217904068090155|
|    min|               21|
|    max|               79|
+-------+-----------------+



Breakdown the data by cateforical data

In [ ]:
ccdefault.groupBy("SEX").count().sort("count", ascending = False).show()
ccdefault.groupBy("EDUCATION").count().sort("count", ascending = False).show()
ccdefault.groupBy("MARRIAGE").count().sort("count", ascending = False).show()

+---+-----+
|SEX|count|
+---+-----+
|  2|18112|
|  1|11888|
+---+-----+

+---------+-----+
|EDUCATION|count|
+---------+-----+
|        2|14030|
|        1|10585|
|        3| 4917|
|        5|  280|
|        4|  123|
|        6|   51|
|        0|   14|
+---------+-----+

+--------+-----+
|MARRIAGE|count|
+--------+-----+
|       2|15964|
|       1|13659|
|       3|  323|
|       0|   54|
+--------+-----+



## 7.3. Train a model to predict the target variable

### 7.3.1. Prepare the data

In [ ]:
renamedccdefault = ccdefault.withColumnRenamed("DEFAULT","LABEL").withColumnRenamed("PAY_0", "PAY_1").drop("ID")
renamedccdefault.show(1)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|LABEL|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|    1|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
only showing top 1 row



Separate the numerical attributes from the categorical attribute

In [ ]:
colLabel = "LABEL"
colCat = ["SEX", "EDUCATION", "MARRIAGE"]
colNum = renamedccdefault.schema.names
colNum.remove(colLabel)
for i in colCat:
    colNum.remove(i)
print(colNum)

['LIMIT_BAL', 'AGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']


Find the columns with missing values in the numerical attributes

In [ ]:
renamedccdefault.select([count(when(col(c).isNull(), c)).alias(c) for c in renamedccdefault.columns]).show()

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|LABEL|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|        0|  0|        0|       0|  0|    0|    0|    0|    0|    0|    0|        0|        0|        0|        0|        0|        0|       0|       0|       0|       0|       0|       0|    0|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+



Scaling

In [ ]:
va = VectorAssembler(inputCols=colNum, outputCol="vector_of_features")

featuredccdefault = va.transform(renamedccdefault)

scaler = StandardScaler().setInputCol("vector_of_features").setOutputCol("scaled_features")
scaledccdefault = scaler.fit(featuredccdefault).transform(featuredccdefault)

scaledccdefault.show(5)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|LABEL|  vector_of_features|     scaled_features|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+
|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|    1|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
|   120000|  2|        2|       2| 26|   -1|    2|  

OneHotEncoder

In [ ]:
encoder = OneHotEncoder(
                        inputCols=["SEX", "EDUCATION", "MARRIAGE"],
                        outputCols=["SEX_e", "EDUCATION_e", "MARRIAGE_e"])
ohccdefault=encoder.fit(renamedccdefault).transform(renamedccdefault)
ohccdefault.show(5)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------+-------------+-------------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|LABEL|        SEX_e|  EDUCATION_e|   MARRIAGE_e|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------+-------------+-------------+
|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|    1|    (2,[],[])|(6,[2],[1.0])|(3,[1],[1.0])|
|   120000|  2|        2|       2| 26|   -1|    2|  

Pipeline

In [ ]:
numPipeline = Pipeline().setStages([va, scaler])
catPipeline = Pipeline().setStages([encoder])
pipeline = Pipeline().setStages([numPipeline, catPipeline])
newccdefault = pipeline.fit(renamedccdefault).transform(renamedccdefault)

newccdefault.show(5)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+-------------+-------------+-------------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|LABEL|  vector_of_features|     scaled_features|        SEX_e|  EDUCATION_e|   MARRIAGE_e|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+-------------+-------------+-------------+
|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|  

Now, use `VectorAssembler` to put all attributes of the final dataset `newccdefault` into a big vector, and call the new column `features`.

In [ ]:
va2 = VectorAssembler().setInputCols(["scaled_features", "SEX_e", "EDUCATION_e", "MARRIAGE_e"]).setOutputCol("features")
dataset = va2.transform(newccdefault).select("features", "LABEL")

dataset.show(5)

+--------------------+-----+
|            features|LABEL|
+--------------------+-----+
|(31,[0,1,2,3,4,5,...|    1|
|(31,[0,1,2,3,7,8,...|    1|
|(31,[0,1,8,9,10,1...|    0|
|(31,[0,1,8,9,10,1...|    0|
|(31,[0,1,2,4,8,9,...|    0|
+--------------------+-----+
only showing top 5 rows



### 7.3.2. Logistic regression

In [ ]:
trainSet, testSet = dataset.randomSplit([0.8, 0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

lr = LogisticRegression().setFeaturesCol("features").setLabelCol("LABEL")

# train the model
lrModel = lr.fit(trainSet)

# make predictions on the test data
predictions = lrModel.transform(testSet)
predictions.select("prediction", "LABEL", "features").show(5)

# evaluate
predictions.groupBy('LABEL','prediction').count().show()
e = BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("LABEL")\
                                   .setMetricName("areaUnderROC")
AUC = e.evaluate(predictions)
print("AUC = " + str(AUC))

+----------+-----+--------------------+
|prediction|LABEL|            features|
+----------+-----+--------------------+
|       1.0|    0|(31,[0,1,2,3,4,5,...|
|       0.0|    0|(31,[0,1,2,3,4,5,...|
|       0.0|    0|(31,[0,1,2,3,4,5,...|
|       1.0|    1|(31,[0,1,2,3,4,5,...|
|       0.0|    0|(31,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

+-----+----------+-----+
|LABEL|prediction|count|
+-----+----------+-----+
|    1|       0.0|  968|
|    0|       0.0| 4620|
|    1|       1.0|  312|
|    0|       1.0|  140|
+-----+----------+-----+

AUC = 0.6071691176470587


### 7.3.3. Decision tree

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier().setFeaturesCol("features").setLabelCol("LABEL")

# train the model
dtModel = dt.fit(trainSet)

# make predictions on the test data
predictions = dtModel.transform(testSet)
predictions.select("prediction", "LABEL", "features").show(5)

# select (prediction, true label) and compute test error
predictions.groupBy('LABEL','prediction').count().show()
e = BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("LABEL")\
                                   .setMetricName("areaUnderROC")
AUC = e.evaluate(predictions)
print("AUC = " + str(AUC))

+----------+-----+--------------------+
|prediction|LABEL|            features|
+----------+-----+--------------------+
|       0.0|    0|(31,[0,1,2,3,4,5,...|
|       0.0|    0|(31,[0,1,2,3,4,5,...|
|       0.0|    0|(31,[0,1,2,3,4,5,...|
|       1.0|    1|(31,[0,1,2,3,4,5,...|
|       0.0|    0|(31,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

+-----+----------+-----+
|LABEL|prediction|count|
+-----+----------+-----+
|    1|       0.0|  809|
|    0|       0.0| 4502|
|    1|       1.0|  471|
|    0|       1.0|  258|
+-----+----------+-----+

AUC = 0.6568835346638655


### 7.3.4. Random forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier().setFeaturesCol("features").setLabelCol("LABEL")

# train the model
rfModel = rf.fit(trainSet)

# make predictions on the test data
predictions = rfModel.transform(testSet)
predictions.select("prediction", "LABEL", "features").show(5)

# select (prediction, true label) and compute test error
predictions.groupBy('LABEL','prediction').count().show()
e = BinaryClassificationEvaluator().setRawPredictionCol("prediction").setLabelCol("LABEL")\
                                   .setMetricName("areaUnderROC")
AUC = e.evaluate(predictions)
print("AUC = " + str(AUC))

+----------+-----+--------------------+
|prediction|LABEL|            features|
+----------+-----+--------------------+
|       1.0|    0|(31,[0,1,2,3,4,5,...|
|       0.0|    0|(31,[0,1,2,3,4,5,...|
|       0.0|    0|(31,[0,1,2,3,4,5,...|
|       1.0|    1|(31,[0,1,2,3,4,5,...|
|       0.0|    0|(31,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

+-----+----------+-----+
|LABEL|prediction|count|
+-----+----------+-----+
|    1|       0.0|  813|
|    0|       0.0| 4541|
|    1|       1.0|  467|
|    0|       1.0|  219|
+-----+----------+-----+

AUC = 0.6594176733193277


### 7.3.5. Defend the choice

In this example, we may think decision tree has the best result and logistic regression has the worset result. And random forest is between them. Actually, their performance are close to each other. Therefore, if we change our data set, we may get different performance result. We should choose the best model based on the data set.

For logistic regression, it is easy to implement; it does not require a lot of compuational time. However, when there are too many features, its perfomance may decrease; it is likely to underfit the data.  

For decision tree, it is easy for people to understand and explain the logic behind it; it is efficient and can be used with large amount of data. However, it is easy to overfit the data; it pays less attention to the relation between different groups of data.

For random forest, it is less likely to overfit the data due to the randomness; it is of high accuracy on a majority of data set. However, when the number of trees is large, it will consmue a lot of time and space to train and predict.

## 7.4. What more would you do with this data? Anything to help you devise a better solution?

First, we can filter the data. Some data are not so relative to the label, we can drop them to dcrease the number of features to improve performance, especially logistic regression.

We can also add cross validation process during training to increase the performance. However, this may cost more time to train and predict.

We may also try some other models like gradient boosting to find the one with best performance on this data set.